In [9]:
import os
import tempfile
import pandas as pd
import csv

from lib.export_google_sheets import export_google_sheets


folder_ids=[
        '1HhIkzwUMBTfStSCyhSRf1SkbQq21hLnM',
        '1NrjWTGweLNoUU1n-EH9yhIU5ZBUHOLs1',
        '1FlCwwi71s1BI4uhOf2CefX85byj1Alo_'
]
DIST_DIR="./raw/sheets/"

try:
    with tempfile.TemporaryDirectory() as temp_dir:
        file_list=export_google_sheets(folder_ids,temp_dir)
        for file in file_list:
            book = pd.ExcelFile(file["export_path"])
            dir_name = os.path.join(DIST_DIR,file['name'])
            os.makedirs(dir_name,exist_ok=True)
            for name in book.sheet_names:
                sheet:pd.DataFrame = book.parse(name)
                sheet.to_csv(os.path.join(dir_name,f"{name}.csv"),encoding="utf_8_sig",index=False,)
except:
    import traceback
    traceback.print_exc()

コミュニケーション能力_0131版_査読 in 2022-02-12T08:43:48.082000+00:00
多職種連携能力_0131版_査読 in 2022-02-12T08:43:02.530000+00:00
科学的探究_0131版_査読 in 2022-02-12T08:42:53.552000+00:00
skip: プロフェッショナリズム_0131版_査読
専門知識に基づいた問題解決能力_0131版_査読 in 2022-02-12T08:33:52.727000+00:00
社会における医療の役割の理解_0131版_査読 in 2022-02-12T08:33:45.967000+00:00
生涯にわたって共に学ぶ姿勢_0131版_査読 in 2022-02-12T08:33:41.658000+00:00
